1. Install and Import Dependencies


In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import requests
from bs4 import BeautifulSoup
import re
import torch

c:\Users\rosha\miniconda3\envs\sentiment\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2. Instantiate Model


In [8]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')




3. Encode and Calculate Sentiment


In [9]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')


In [10]:
tokenizer.decode(tokens[0])

'[CLS] it was good but couldve been better. great [SEP]'

In [11]:
tokens

tensor([[  101, 10197, 10140, 12050, 10502, 12296, 10598, 10662, 16197,   119,
         11838,   102]])

In [12]:
result = model(tokens)


In [13]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [14]:
result.logits


tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [15]:
int(torch.argmax(result.logits))+1

4

4. Collect Reviews


In [16]:
r = requests.get('https://www.yelp.com/biz/tycoon-kitchen-san-francisco-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

ConnectionError: HTTPSConnectionPool(host='www.yelp.com', port=443): Max retries exceeded with url: /biz/tycoon-kitchen-san-francisco-2 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000023B61AE0D90>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
r

<Response [200]>

In [ ]:
r.text

'<!DOCTYPE html><html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/\x08no-js\x08/,"js");</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="Content-Language" content="en-US" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel="mask-icon" sizes="any" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" content="#FF1A1A"><link rel="shortcut icon" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/dcfe403147fc/assets/img/logos/favicon.ico"><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>\n            window.yelp = window.yelp || {};\

In [ ]:
reviews

['Freshly opened store in this neighborhood and a welcome addition here! While tiny, it has full service and some limited seating indoors and outside. Food came out relatively fast. It was nicely decorated and well presented, and service was friendly. I had the salmon benedict. The salmon was fresh, and I was able to substitute fresh fruit for the potatoes. Plentiful portion, with enough for a takeaway. Will try the taro float next time! They are also open on weekends, which is a much needed option in this neighborhood.',
 'Thank you for your kind words about our newly opened restaurant! We are thrilled to hear that you enjoyed your experience with us and appreciate your feedback. Our team takes pride in offering full service and fast food, and we are glad to hear that you found our salmon benedict to be fresh and delicious. We are happy to accommodate substitutions and are glad that you enjoyed the option to substitute fresh fruit for the potatoes. We are also excited for you to try o

5. Load Reviews into DataFrame and Score


In [18]:
import pandas as pd
import numpy as np

In [19]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

NameError: name 'reviews' is not defined

In [20]:
df['review'].iloc[0]


KeyError: 'review'

In [21]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[1])


5

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))


In [ ]:
df

,review,sentiment
0,Freshly opened store in this neighborhood and ...,5
1,Thank you for your kind words about our newly ...,5
2,I started a new job in the area a month ago. J...,3
3,Thank you for choosing Tycoon Kitchen as your ...,5
4,Apparently the outside kiosk doesn't work for ...,3
5,For Drunken Noodles w/ Beef We're pleased to h...,4
6,Visited Tycoon Kitchen on a weekday for lunch ...,4
7,Thank you for taking the time to share your ex...,5
8,Spring rollsPad saew Egg plant tofu First time...,4
9,"HI Brian, It's great to hear that despite the ...",5


In [ ]:
df['review'].iloc[12]


"My new favorite brunch spot right under the Salesforce Tower! It's a little tucked away but if you walk down Minna or Natoma, you'll see the first floor area of shops including Tycoon Kitchen. We came in for a weekend brunch. There wasn't a wait but the space is very small with only a few indoor tables and a few outdoor tables set up as well. I think this is more of a takeout place but of course if you need an area to sit to eat, Salesforce park is right upstairs!The thai tea french toast caught my eye and it was every bit as flavorful and delicious as I imagined. The thai tea flavor was strong but it wasn't too sweet of an entree. I appreciated the side of fresh fruit too. For $14 it was magnificent. You can also pay more for a combo to add eggs and meat but that sounded like too much for me at the time. My bf got the egg and cheese breakfast sandwich which was also tasty in its simplicity. It came with a huge side of fries which we didn't expect, but they were very good too! Crispy 

Sentiment analysis on youtube comments


In [11]:
import youtube_comment_scraper_python

Starting DataKund...


In [2]:
from youtube_comment_scraper_python import *
import requests
import time
import pandas as pd

In [3]:
url = "https://www.youtube.com/watch?v=ScCt7Lnz3mU&ab_channel=LahariMusic-TSeries"
youtube.open(url)
fullcomments = []
for i in range(0,1):
    result = youtube.video_comments()
    data = result['body']
    fullcomments.extend(data)

Progress:  65%|██████▌   | 130.8/200 [00:43<00:47,  1.47it/s]

Progress: 100%|██████████| 200.0/200 [00:47<00:00,  4.21it/s]


In [ ]:
df = pd.DataFrame(fullcomments)


,Comment,Likes,Time,UserLink,user
0,#Adipurush Prabhas. Jai Shri Ram,5.4K,8 months ago,https://www.youtube.com/channel/UCnSqxrSfo1sK4...,
1,Who came here after watching trailer to see di...,172,1 month ago,NaN,
2,"Adipurush SriRam,Adikavya Ramayan are eternal!...",4,3 weeks ago,NaN,
3,My dear telugu friends don't worry about this ...,133,1 month ago,NaN,
4,Who came after seeing trailer,137,1 month ago,NaN,


In [22]:
df['sentiment'] = df['Comment'].apply(lambda x: sentiment_score(x[:512]))


In [23]:
df

,Comment,Likes,Time,UserLink,user,sentiment
0,#Adipurush Prabhas. Jai Shri Ram,5.4K,8 months ago,https://www.youtube.com/channel/UCnSqxrSfo1sK4...,,5
1,Who came here after watching trailer to see di...,172,1 month ago,NaN,,1
2,"Adipurush SriRam,Adikavya Ramayan are eternal!...",4,3 weeks ago,NaN,,5
3,My dear telugu friends don't worry about this ...,133,1 month ago,NaN,,5
4,Who came after seeing trailer,137,1 month ago,NaN,,1
5,Being Telugu movie lover i am always mesmerize...,9,1 month ago (edited),NaN,,5
6,How many here prabhas anna fans?? ️,4,3 days ago,NaN,,1
7,Adipurush prabhas sir voice,85,8 months ago,https://www.youtube.com/channel/UCL0pM5-vrLp3O...,,5
8,Huge Respect to Prabhas for making film for kids.,1K,8 months ago,NaN,,5
9,High demand for kids show TV's to purchasing t...,64,8 months ago,NaN,,5


In [27]:
df.drop(['Likes','Time','UserLink','user'],axis=1,inplace = True)

In [24]:
df.to_csv('a.csv')

In [4]:
n=int(input())
k=int(input())
l=list(range(n))
a=len(l)
print(l)
i=0
while(len(l)!=1):
    x=len(l)
    l.pop((i+k-1)%x)
    print(l)
    i=(i+k-1)%x
    print(i)

print(l[0])


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
0
[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
1
[1, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
3
[1, 3, 4, 6, 7, 8, 10, 11, 12, 13, 14, 15]
6
[1, 3, 4, 6, 7, 8, 10, 11, 12, 13, 15]
10
[1, 3, 4, 6, 8, 10, 11, 12, 13, 15]
4
[3, 4, 6, 8, 10, 11, 12, 13, 15]
0
[3, 4, 6, 8, 10, 11, 12, 15]
7
[3, 4, 6, 8, 10, 11, 12]
7
[3, 4, 8, 10, 11, 12]
2
[4, 8, 10, 11, 12]
0
[4, 10, 11, 12]
1
[4, 11, 12]
1
[4, 11]
2
[11]
0
11


In [4]:
l

[0, 1, 2, 3]